In [1]:
import requests
import json
import pandas as pd
import numpy as np
#import time
#from datetime import datetime
#from datetime import timedelta
import pickle

from keras.models import Sequential

Using TensorFlow backend.
C:\Users\tomot\anaconda3\envs\lstm\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomot\anaconda3\envs\lstm\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomot\anaconda3\envs\lstm\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomot\anaconda3\envs\lstm\lib\site-packag

In [2]:
# 2019/02/01～の情報を取得（UTCとの時差は9時間）（close_timeは日本時間で午前9時）
res = requests.get('https://api.cryptowat.ch/markets/bitflyer/btcjpy/ohlc?periods=86400&after=1548979200')
data = json.loads(res.text)
daily_data = pd.DataFrame(data['result']['86400'], columns=['close_time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'quote_volume'])
daily_data['date'] = pd.to_datetime(daily_data['close_time'], unit='s')
daily_data = daily_data[:-1] # 最終行を削除（取得した時点の情報が次の日の情報となっているため）

In [3]:
with open("lstm_model.pickle", "rb") as f:
    model = pickle.load(f)

In [4]:
period = 5
data = daily_data['close_price']

input_tensor = []
label_tensor = []
mean_list = []
std_list = []
data_len = len(data)    #総データ数
    
#変数とラベルの生成
for i in range(0, data_len - period, 1):
    window = data.values[i:i + period]
    mean = np.mean(window)
    std = np.std(window)
    mean_list.append(mean)
    std_list.append(std)
    input_tensor.append((window-mean) / std)
    label_tensor.append((data.values[i + period]-mean) / std)
        
input_tensor = np.array(input_tensor).reshape(len(data)-period, period, 1)
label_tensor = np.array(label_tensor).reshape(len(data)-period, 1)
mean_list = np.array(mean_list)
std_list = np.array(std_list)

In [5]:
date = daily_data['date'][5:].reset_index(drop=True)

In [6]:
pred = model.predict(input_tensor)
pred = pred.reshape(-1)
pred = pred*std_list + mean_list
label_tensor = label_tensor.reshape(-1)
real = label_tensor*std_list + mean_list
past_df = pd.DataFrame({'date':date, 'pred':pred, 'real':real})
past_df.tail()

,date,pred,real
742,2021-02-17,5.076019e+06,5205481.0
743,2021-02-18,5.234057e+06,5526975.0
744,2021-02-19,5.605767e+06,5456000.0
745,2021-02-20,5.514918e+06,5910646.0
746,2021-02-21,5.998294e+06,5910115.0


In [15]:
a=past_df['date'].iloc[0]
a

Timestamp('2019-02-06 00:00:00')

In [2]:
from assets import models

from assets.database import db_session


In [3]:
from assets.database import init_db

init_db()

2021-02-22 19:20:14,704 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-22 19:20:14,709 INFO sqlalchemy.engine.base.Engine ()
2021-02-22 19:20:14,717 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-22 19:20:14,723 INFO sqlalchemy.engine.base.Engine ()
2021-02-22 19:20:14,731 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("data")
2021-02-22 19:20:14,736 INFO sqlalchemy.engine.base.Engine ()
2021-02-22 19:20:14,746 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("data")
2021-02-22 19:20:14,753 INFO sqlalchemy.engine.base.Engine ()
2021-02-22 19:20:14,760 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE data (
	id INTEGER NOT NULL, 
	date DATETIME, 
	real_price FLOAT, 
	pred_price FLOAT, 
	timestamp DATETIME, 
	PRIMARY KEY (id)
)


2021-02-22 19:20:14,764 INFO sqlalchemy.engine.base.Engine ()
2021-02-22 19:20:14,793 INFO sqlalchemy.engine.base.Engine COMMIT


In [17]:
row = models.Data(date = past_df['date'].iloc[0], real_price = past_df['real'].iloc[0], pred_price = past_df['pred'].iloc[0])
db_session.add(row)

In [18]:
db_session.commit()

2021-02-22 00:28:57,496 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-22 00:28:57,502 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:28:57,512 INFO sqlalchemy.engine.base.Engine ('2019-02-06 00:00:00.000000', 376384.0, 374529.5314869114, '2021-02-22 00:23:56.804582')
2021-02-22 00:28:57,524 INFO sqlalchemy.engine.base.Engine COMMIT


In [4]:
import datetime
date = datetime.date.today()
row = models.Data(date=date, real_price=30000.0, pred_price=20000.0)

In [5]:
db_session.add(row)

In [6]:
db_session.commit()

2021-02-22 00:17:48,525 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-22 00:17:48,535 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:17:48,543 INFO sqlalchemy.engine.base.Engine ('2021-02-22 00:00:00.000000', 30000.0, 20000.0, '2021-02-22 00:17:05.779172')
2021-02-22 00:17:48,556 INFO sqlalchemy.engine.base.Engine COMMIT


In [8]:
db_session.query(models.Data).all()[0].date

2021-02-22 00:19:22,473 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-22 00:19:22,481 INFO sqlalchemy.engine.base.Engine SELECT data.id AS data_id, data.date AS data_date, data.real_price AS data_real_price, data.pred_price AS data_pred_price, data.timestamp AS data_timestamp 
FROM data
2021-02-22 00:19:22,488 INFO sqlalchemy.engine.base.Engine ()


datetime.datetime(2021, 2, 22, 0, 0)

In [20]:

for index, _past_df in past_df.iterrows():
    row = models.Data(date = _past_df['date'], real_price = _past_df['real'], pred_price = _past_df['pred'])
    db_session.add(row)
    
db_session.commit()

2021-02-22 00:29:55,595 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-22 00:29:55,615 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:55,623 INFO sqlalchemy.engine.base.Engine ('2019-02-06 00:00:00.000000', 376384.0, 374529.5314869114, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:55,640 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:55,647 INFO sqlalchemy.engine.base.Engine ('2019-02-07 00:00:00.000000', 369767.0, 376527.961247868, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:55,654 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:55,662 INFO sqlalchemy.engine.base.Engine ('2019-02-08 00:00:00.000000', 368506.0, 371621.32793994085, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:55,669 INFO sqlalchemy.engine.base.Engine INSE

2021-02-22 00:29:56,415 INFO sqlalchemy.engine.base.Engine ('2019-04-03 00:00:00.000000', 545469.0, 461938.734322117, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:56,427 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:56,431 INFO sqlalchemy.engine.base.Engine ('2019-04-04 00:00:00.000000', 555300.0, 556856.0345419074, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:56,439 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:56,447 INFO sqlalchemy.engine.base.Engine ('2019-04-05 00:00:00.000000', 548000.0, 575529.1077154774, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:56,457 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:56,462 INFO sqlalchemy.engine.base.Engine ('2019-04-06 00:00:00.000000', 562532.0, 555409.3112297106, '2021-02-22 00:23:56.

2021-02-22 00:29:57,288 INFO sqlalchemy.engine.base.Engine ('2019-05-30 00:00:00.000000', 948273.0, 958541.2596675691, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:57,297 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:57,305 INFO sqlalchemy.engine.base.Engine ('2019-05-31 00:00:00.000000', 907386.0, 948912.8186389765, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:57,312 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:57,320 INFO sqlalchemy.engine.base.Engine ('2019-06-01 00:00:00.000000', 928180.0, 921267.9618989141, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:57,329 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:57,335 INFO sqlalchemy.engine.base.Engine ('2019-06-02 00:00:00.000000', 925022.0, 925632.324737471, '2021-02-22 00:23:56.

2021-02-22 00:29:58,139 INFO sqlalchemy.engine.base.Engine ('2019-07-25 00:00:00.000000', 1056426.0, 1084237.2922403421, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:58,148 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:58,156 INFO sqlalchemy.engine.base.Engine ('2019-07-26 00:00:00.000000', 1074893.0, 1063675.4250667577, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:58,166 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:58,172 INFO sqlalchemy.engine.base.Engine ('2019-07-27 00:00:00.000000', 1070121.0, 1075716.1940893608, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:58,179 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:58,189 INFO sqlalchemy.engine.base.Engine ('2019-07-28 00:00:00.000000', 1030983.0, 1076124.2435872427, '2021-02-22 

2021-02-22 00:29:58,999 INFO sqlalchemy.engine.base.Engine ('2019-09-19 00:00:00.000000', 1100344.0, 1104145.058386488, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:59,006 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:59,013 INFO sqlalchemy.engine.base.Engine ('2019-09-20 00:00:00.000000', 1108999.0, 1101486.34695307, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:59,020 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:59,026 INFO sqlalchemy.engine.base.Engine ('2019-09-21 00:00:00.000000', 1093999.0, 1108474.6666829232, '2021-02-22 00:23:56.804582')
2021-02-22 00:29:59,034 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:29:59,042 INFO sqlalchemy.engine.base.Engine ('2019-09-22 00:00:00.000000', 1074107.0, 1097537.6456854003, '2021-02-22 00:

...

2021-02-22 00:30:03,464 INFO sqlalchemy.engine.base.Engine ('2020-06-28 00:00:00.000000', 969100.0, 987615.8641710741, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:03,470 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:03,481 INFO sqlalchemy.engine.base.Engine ('2020-06-29 00:00:00.000000', 979187.0, 975117.3283144784, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:03,486 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:03,492 INFO sqlalchemy.engine.base.Engine ('2020-06-30 00:00:00.000000', 990000.0, 978103.3961020133, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:03,499 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:03,504 INFO sqlalchemy.engine.base.Engine ('2020-07-01 00:00:00.000000', 987501.0, 991424.8679882259, '2021-02-22 00:23:56

2021-02-22 00:30:04,300 INFO sqlalchemy.engine.base.Engine ('2020-08-23 00:00:00.000000', 1237485.0, 1232982.6150094364, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:04,307 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:04,314 INFO sqlalchemy.engine.base.Engine ('2020-08-24 00:00:00.000000', 1235514.0, 1235852.9826177023, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:04,318 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:04,328 INFO sqlalchemy.engine.base.Engine ('2020-08-25 00:00:00.000000', 1246899.0, 1236470.0208838838, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:04,336 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:04,340 INFO sqlalchemy.engine.base.Engine ('2020-08-26 00:00:00.000000', 1208500.0, 1249948.1975377447, '2021-02-22 

2021-02-22 00:30:05,134 INFO sqlalchemy.engine.base.Engine ('2020-10-18 00:00:00.000000', 1197535.0, 1198626.526763807, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:05,143 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:05,150 INFO sqlalchemy.engine.base.Engine ('2020-10-19 00:00:00.000000', 1213571.0, 1197626.7134840665, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:05,157 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:05,164 INFO sqlalchemy.engine.base.Engine ('2020-10-20 00:00:00.000000', 1237405.0, 1214436.0722789657, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:05,173 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:05,178 INFO sqlalchemy.engine.base.Engine ('2020-10-21 00:00:00.000000', 1256584.0, 1244279.4014660511, '2021-02-22 0

2021-02-22 00:30:05,951 INFO sqlalchemy.engine.base.Engine ('2020-12-13 00:00:00.000000', 1957880.0, 1886875.8507260678, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:05,957 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:05,966 INFO sqlalchemy.engine.base.Engine ('2020-12-14 00:00:00.000000', 1988187.0, 1953257.7656325772, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:05,973 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:05,979 INFO sqlalchemy.engine.base.Engine ('2020-12-15 00:00:00.000000', 2000099.0, 2008440.7503635222, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:05,986 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:05,990 INFO sqlalchemy.engine.base.Engine ('2020-12-16 00:00:00.000000', 2011544.0, 2014081.4163231188, '2021-02-22 

2021-02-22 00:30:06,773 INFO sqlalchemy.engine.base.Engine ('2021-02-07 00:00:00.000000', 4131757.0, 4073272.9961310425, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:06,782 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:06,786 INFO sqlalchemy.engine.base.Engine ('2021-02-08 00:00:00.000000', 4097000.0, 4177361.367391569, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:06,794 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:06,800 INFO sqlalchemy.engine.base.Engine ('2021-02-09 00:00:00.000000', 4875000.0, 4116769.5886623785, '2021-02-22 00:23:56.804582')
2021-02-22 00:30:06,810 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 00:30:06,814 INFO sqlalchemy.engine.base.Engine ('2021-02-10 00:00:00.000000', 4874233.0, 4974096.493954754, '2021-02-22 00

In [4]:
def read_data():
    from assets import models

    # 2019/02/01～の情報を取得（UTCとの時差は9時間）（close_timeは日本時間で午前9時）
    res = requests.get('https://api.cryptowat.ch/markets/bitflyer/btcjpy/ohlc?periods=86400&after=1548979200')
    data = json.loads(res.text)
    daily_data = pd.DataFrame(data['result']['86400'], columns=['close_time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'quote_volume'])
    daily_data['date'] = pd.to_datetime(daily_data['close_time'], unit='s')
    daily_data = daily_data[:-1] # 最終行を削除（取得した時点の情報が次の日の情報となっているため）

    period = 5
    data = daily_data['close_price']

    input_tensor = []
    label_tensor = []
    mean_list = []
    std_list = []
    data_len = len(data)    #総データ数

    #変数とラベルの生成
    for i in range(0, data_len - period, 1):
        window = data.values[i:i + period]
        mean = np.mean(window)
        std = np.std(window)
        mean_list.append(mean)
        std_list.append(std)
        input_tensor.append((window-mean) / std)
        label_tensor.append((data.values[i + period]-mean) / std)

    input_tensor = np.array(input_tensor).reshape(len(data)-period, period, 1)
    label_tensor = np.array(label_tensor).reshape(len(data)-period, 1)
    mean_list = np.array(mean_list)
    std_list = np.array(std_list)

    date = daily_data['date'][5:].reset_index(drop=True)

    with open("lstm_model.pickle", "rb") as f:
        model = pickle.load(f)

    pred = model.predict(input_tensor)
    pred = pred.reshape(-1)
    pred = pred*std_list + mean_list
    label_tensor = label_tensor.reshape(-1)
    real = label_tensor*std_list + mean_list
    past_df = pd.DataFrame({'date':date, 'pred':pred, 'real':real})

    for index, _past_df in past_df.iterrows():
        row = models.Data(date = _past_df['date'], real_price = _past_df['real'], pred_price = _past_df['pred'])
        db_session.add(row)

    db_session.commit()

In [5]:
read_data()

2021-02-22 19:20:57,363 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-22 19:20:57,383 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:20:57,388 INFO sqlalchemy.engine.base.Engine ('2019-02-06 00:00:00.000000', 376384.0, 374529.5314869114, '2021-02-22 19:19:46.857845')
2021-02-22 19:20:57,402 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:20:57,407 INFO sqlalchemy.engine.base.Engine ('2019-02-07 00:00:00.000000', 369767.0, 376527.961247868, '2021-02-22 19:19:46.857845')
2021-02-22 19:20:57,414 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:20:57,421 INFO sqlalchemy.engine.base.Engine ('2019-02-08 00:00:00.000000', 368506.0, 371621.32793994085, '2021-02-22 19:19:46.857845')
2021-02-22 19:20:57,429 INFO sqlalchemy.engine.base.Engine INSE

2021-02-22 19:20:58,044 INFO sqlalchemy.engine.base.Engine ('2019-04-03 00:00:00.000000', 545469.0, 461938.734322117, '2021-02-22 19:19:46.857845')
2021-02-22 19:20:58,052 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:20:58,058 INFO sqlalchemy.engine.base.Engine ('2019-04-04 00:00:00.000000', 555300.0, 556856.0345419074, '2021-02-22 19:19:46.857845')
2021-02-22 19:20:58,063 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:20:58,068 INFO sqlalchemy.engine.base.Engine ('2019-04-05 00:00:00.000000', 548000.0, 575529.1077154774, '2021-02-22 19:19:46.857845')
2021-02-22 19:20:58,073 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:20:58,077 INFO sqlalchemy.engine.base.Engine ('2019-04-06 00:00:00.000000', 562532.0, 555409.3112297106, '2021-02-22 19:19:46.

2021-02-22 19:20:58,817 INFO sqlalchemy.engine.base.Engine ('2019-05-30 00:00:00.000000', 948273.0, 958541.2596675691, '2021-02-22 19:19:46.857845')
2021-02-22 19:20:58,823 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:20:58,829 INFO sqlalchemy.engine.base.Engine ('2019-05-31 00:00:00.000000', 907386.0, 948912.8186389765, '2021-02-22 19:19:46.857845')
2021-02-22 19:20:58,837 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:20:58,844 INFO sqlalchemy.engine.base.Engine ('2019-06-01 00:00:00.000000', 928180.0, 921267.9618989141, '2021-02-22 19:19:46.857845')
2021-02-22 19:20:58,856 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:20:58,864 INFO sqlalchemy.engine.base.Engine ('2019-06-02 00:00:00.000000', 925022.0, 925632.324737471, '2021-02-22 19:19:46.

2021-02-22 19:20:59,603 INFO sqlalchemy.engine.base.Engine ('2019-07-25 00:00:00.000000', 1056426.0, 1084237.2922403421, '2021-02-22 19:19:46.857845')
2021-02-22 19:20:59,609 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:20:59,613 INFO sqlalchemy.engine.base.Engine ('2019-07-26 00:00:00.000000', 1074893.0, 1063675.4250667577, '2021-02-22 19:19:46.857845')
2021-02-22 19:20:59,617 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:20:59,625 INFO sqlalchemy.engine.base.Engine ('2019-07-27 00:00:00.000000', 1070121.0, 1075716.1940893608, '2021-02-22 19:19:46.857845')
2021-02-22 19:20:59,632 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:20:59,637 INFO sqlalchemy.engine.base.Engine ('2019-07-28 00:00:00.000000', 1030983.0, 1076124.2435872427, '2021-02-22 

2021-02-22 19:21:00,317 INFO sqlalchemy.engine.base.Engine ('2019-09-19 00:00:00.000000', 1100344.0, 1104145.058386488, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:00,324 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:00,330 INFO sqlalchemy.engine.base.Engine ('2019-09-20 00:00:00.000000', 1108999.0, 1101486.34695307, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:00,337 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:00,341 INFO sqlalchemy.engine.base.Engine ('2019-09-21 00:00:00.000000', 1093999.0, 1108474.6666829232, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:00,347 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:00,353 INFO sqlalchemy.engine.base.Engine ('2019-09-22 00:00:00.000000', 1074107.0, 1097537.6456854003, '2021-02-22 19:

...

2021-02-22 19:21:03,966 INFO sqlalchemy.engine.base.Engine ('2020-06-28 00:00:00.000000', 969100.0, 987615.8641710741, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:03,970 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:03,975 INFO sqlalchemy.engine.base.Engine ('2020-06-29 00:00:00.000000', 979187.0, 975117.3283144784, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:03,982 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:03,987 INFO sqlalchemy.engine.base.Engine ('2020-06-30 00:00:00.000000', 990000.0, 978103.3961020133, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:03,992 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:03,995 INFO sqlalchemy.engine.base.Engine ('2020-07-01 00:00:00.000000', 987501.0, 991424.8679882259, '2021-02-22 19:19:46

2021-02-22 19:21:04,703 INFO sqlalchemy.engine.base.Engine ('2020-08-23 00:00:00.000000', 1237485.0, 1232982.6150094364, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:04,709 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:04,716 INFO sqlalchemy.engine.base.Engine ('2020-08-24 00:00:00.000000', 1235514.0, 1235852.9826177023, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:04,726 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:04,735 INFO sqlalchemy.engine.base.Engine ('2020-08-25 00:00:00.000000', 1246899.0, 1236470.0208838838, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:04,743 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:04,751 INFO sqlalchemy.engine.base.Engine ('2020-08-26 00:00:00.000000', 1208500.0, 1249948.1975377447, '2021-02-22 

2021-02-22 19:21:05,565 INFO sqlalchemy.engine.base.Engine ('2020-10-18 00:00:00.000000', 1197535.0, 1198626.526763807, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:05,573 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:05,578 INFO sqlalchemy.engine.base.Engine ('2020-10-19 00:00:00.000000', 1213571.0, 1197626.7134840665, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:05,587 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:05,590 INFO sqlalchemy.engine.base.Engine ('2020-10-20 00:00:00.000000', 1237405.0, 1214436.0722789657, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:05,597 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:05,603 INFO sqlalchemy.engine.base.Engine ('2020-10-21 00:00:00.000000', 1256584.0, 1244279.4014660511, '2021-02-22 1

2021-02-22 19:21:06,195 INFO sqlalchemy.engine.base.Engine ('2020-12-13 00:00:00.000000', 1957880.0, 1886875.8507260678, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:06,203 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:06,210 INFO sqlalchemy.engine.base.Engine ('2020-12-14 00:00:00.000000', 1988187.0, 1953257.7656325772, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:06,216 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:06,222 INFO sqlalchemy.engine.base.Engine ('2020-12-15 00:00:00.000000', 2000099.0, 2008440.7503635222, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:06,229 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:06,236 INFO sqlalchemy.engine.base.Engine ('2020-12-16 00:00:00.000000', 2011544.0, 2014081.4163231188, '2021-02-22 

2021-02-22 19:21:06,917 INFO sqlalchemy.engine.base.Engine ('2021-02-07 00:00:00.000000', 4131757.0, 4073272.9961310425, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:06,923 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:06,929 INFO sqlalchemy.engine.base.Engine ('2021-02-08 00:00:00.000000', 4097000.0, 4177361.367391569, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:06,936 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:06,942 INFO sqlalchemy.engine.base.Engine ('2021-02-09 00:00:00.000000', 4875000.0, 4116769.5886623785, '2021-02-22 19:19:46.857845')
2021-02-22 19:21:06,950 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, real_price, pred_price, timestamp) VALUES (?, ?, ?, ?)
2021-02-22 19:21:06,959 INFO sqlalchemy.engine.base.Engine ('2021-02-10 00:00:00.000000', 4874233.0, 4974096.493954754, '2021-02-22 19